In [1]:
from __future__ import print_function

import sys

from pyspark import SparkContext
from pyspark.streaming import StreamingContext
from textblob import TextBlob
from textblob.sentiments import NaiveBayesAnalyzer
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer 


In [2]:
sc.setLogLevel("ERROR")
ssc = StreamingContext(sc, 5) 

In [3]:
#ssc.checkpoint("Twitter_streaming")
ssc.remember(60)
dbutils.fs.rm("dbfs:/streaming/checkpoint/100", True)
ssc.checkpoint("dbfs:/streaming/checkpoint/100")

In [4]:
stream = ssc.socketTextStream("localhost", 10002)

In [5]:
print(type(stream))

<class 'pyspark.streaming.dstream.DStream'>

In [6]:
def tuits(tweets):
  for tweet in tweets:
    print(tweet)
    print(TextBlob(tweet).sentiment)
    textsid_obj = SentimentIntensityAnalyzer() 
    sentiment_dict = textsid_obj.polarity_scores(tweet) 
    print("\n")
    print("Sentiment ( " + str(sentiment_dict['compound']) + " )")
    if sentiment_dict['compound'] >= 0.05 : 
        print("Positive") 
    elif sentiment_dict['compound'] <= - 0.05 : 
        print("Negative") 
    else : 
        print("Neutral")
        print("\n")

In [7]:

stream.foreachRDD(lambda line : tuits(line.collect()))

ssc.start()
ssc.awaitTermination()